In [5]:
import numpy as np


def r(j, k, velocities):
    m = np.median(velocities[j:k])
    return m, np.linalg.norm(velocities[j:k] - m, ord=1)

def a(j, k, accelerations):
    m = np.median(accelerations[j:k-1])
    return m, np.linalg.norm(accelerations[j:k-1] - m, ord=1)

def fit_segment(j, k, acc, vel): # fits the segment from j to k and returns the type, error, and value
    (r_val, r_err), (a_val, a_err) = r(j, k, vel), a(j, k, acc)
    if a_err < r_err:
        return 'accelerating', a_err, a_val
    else:
        return 'regular', r_err, r_val

def find_optimal_partition(distances, λ=10):
    n = len(distances)
    velocities = np.array([y - x for (x, y) in zip(distances, distances[1:])])
    accelerations = np.array([y - x for (x, y) in zip(velocities, velocities[1:])])
    
    D = np.zeros(n)
    partitions = [[0], [0, 1]]
    partition_segments = [[], []]

    for i in range(2, n):
        min_k = 0
        segment = fit_segment(0, i, accelerations, velocities)
        min_cost = segment[1] + λ
        for k in range(2, i - 1):
            seg = fit_segment(k, i, accelerations, velocities)
            cost = D[k] + seg[1] + λ
            if cost < min_cost:
                min_cost, min_k, segment = cost, k, seg
        D[i] = min_cost
        partitions.append(partitions[min_k] + [i])
        partition_segments.append(partition_segments[min_k] + [segment])

    best_partition = zip(zip(partitions[-1], partitions[-1][1:]), partition_segments[-1])
    return [dict(start=intvl[0], end=intvl[1], type=seg[0], error=seg[1], value=seg[2]) 
            for (intvl, seg) in best_partition]

In [7]:
# [0:5] regular with period about 60, 
# [5:13] constant acceleration of about 12, 
# [13:20] constant deceleration of about 8,
# [20:24] regular with period about 30

distances = np.array([0, 60.3, 121.1, 180.9, 241.9, 301.0, 307.3, 331.3, 366.9, 417.0, 478.7, 550.8, 632.6, 726.5, 831.6, 928.7, 1018.0, 1099.1, 1172.3, 1237.5, 1294.7, 1325.0, 1354.6, 1384.5, 1414.9])

find_optimal_partition(distances)

[{'start': 0,
  'end': 5,
  'type': 'regular',
  'error': 2.8999999999999915,
  'value': 60.3},
 {'start': 5,
  'end': 13,
  'type': 'accelerating',
  'error': 12.599999999999909,
  'value': 11.599999999999966},
 {'start': 13,
  'end': 20,
  'type': 'accelerating',
  'error': 0.5000000000001137,
  'value': -8.0},
 {'start': 20,
  'end': 24,
  'type': 'regular',
  'error': 1.2000000000000455,
  'value': 30.100000000000023}]

In [35]:
# First attempt at a better acceleration metric. Returns two values, representing the inconsistency 
# and magnitude of acceleration.

# TODO: Third value of metric could be the area under velocity curve of acceleration vs total area 
# under curve, which represents the degree to which the sequence is accelerating.

# TODO: Should we be scaling by size of sequence or length in time? Which one results in a more
# useful metric?


def helper(part):
    weights = [seg['end'] - seg['start'] for seg in part] # scaling by sequence length
            # [distances[seg['end']] - distances[seg['start']] for seg in part]
    values = [seg['value'] if seg['type'] == 'accelerating' else 0 for seg in part]
    average = np.average(values, weights=weights) # TODO: weighted median instead of mean?
    variance = np.average((values - average)**2, weights=weights) # TODO: unbiased variance for small sequences?
    
    return np.sqrt(variance), average

def acceleration_metric(distances, λ=0):
    opt_part = find_optimal_partition(distances, λ=λ)
    return helper(opt_part)

In [36]:
# [0:4] acceleration 1
# [3:8] acceleration 2
# [8:13] regular 12 (acceleration 0)

distances = [0, 1, 3, 6, 10, 16, 24, 34, 46, 58, 70, 82, 94, 106]

acceleration_metric(distances)

(0.8634593969478326, 0.8461538461538461)